In [22]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

This program inputs a list of features about the users, and movies and the ratings the users have given to movies and trains neural networks for a recommender system.

In [19]:
df_y_train = pd.read_csv("C:/Users/Fnu Aymen/Documents/Machine Learning Data/contrec/content_y_train.csv",header=None)

# Convert the DataFrame containing the training targets to a NumPy ndarray
y_train = df_y_train.to_numpy()


#contains user id, rating count, rating average, and per genre rating average
df_user_train = pd.read_csv("C:/Users/Fnu Aymen/Documents/Machine Learning Data/contrec/content_user_train.csv",header=None)

#Contains movie id, year of release, average rating and then 14 one-hot encoded genres
df_item_train = pd.read_csv("C:/Users/Fnu Aymen/Documents/Machine Learning Data/contrec/content_item_train.csv",header=None)

# Convert the DataFrame containing the training targets to a NumPy ndarray
y_train = df_y_train.to_numpy()
user_train = df_user_train.to_numpy()
item_train = df_item_train.to_numpy()

# Remove the first three columns from the NumPy array `user_train`
user_train = user_train[:, 2:]
item_train = item_train[:,2:]


print(user_train.shape)
print(item_train.shape)

(50884, 15)
(50884, 15)


In [24]:
#Scale the datasets

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))

In [27]:
#Split the datasets into training and testing sets
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)

num_user_features = user_train.shape[1]
num_item_features = item_train.shape[1]



In [32]:
#Define the two neural networks,one for the user and one for the item.

tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32, activation='linear')
    
])

item_NN = tf.keras.models.Sequential([
      
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32, activation='linear')
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features,))
vu = user_NN(input_user)
#vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features,))
vm = item_NN(input_item)
#vm = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

tf.random.set_seed(1)
model.fit([user_train, item_train], y_train, epochs=30)


Epoch 1/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 0.2677
Epoch 2/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.1235
Epoch 3/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.1214
Epoch 4/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 0.1213
Epoch 5/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.1208
Epoch 6/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.1209
Epoch 7/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.1223
Epoch 8/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.1182
Epoch 9/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.1179
Epoch 10/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.1170
Epoch 11/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.1173
Epoch 12/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.1167
Epoch 13/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.1163
Epoch 14/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.1169
Epoch 15/30
1273/1273 ━━━━━

In [33]:
#Test
model.evaluate([user_test, item_test], y_test)
#Number is pretty close to training loss indicating no overfitting

319/319 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1196


0.11538615822792053

In [ ]:
#To make a prediction we use


model.predict([custom_user,custom_movie])

#where custom user and custom movie are custom user_feature and item_feature vectors which could tell you the predicted rating.
#once the model is trained we can get predicted scores of movies for a user and sort them accordingly

